In [3]:
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from eth_rpc import PrivateKeyWallet
from emp_orderly.utils import from_address
from emp_orderly import (
    Strategy, EmpOrderly,
    crossover, plot_heatmaps,
    EMA, SMA, SLOPE, CHOP,
    EmpyrealOrderlySDK,
)
from emp_orderly_types import PerpetualAssetType, Interval, OrderType

# Load environment variables
load_dotenv()

# Initialize wallet and SDK
wallet = PrivateKeyWallet.create_new()
orderly_id = from_address(wallet.address)
sdk = EmpyrealOrderlySDK(pvt_hex=wallet.private_key, account_id=orderly_id, is_testnet=True)

# Manual RSI Implementation
def RSI(close, period=14):
    delta = np.diff(close)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.convolve(gain, np.ones((period,)) / period, mode='valid')
    avg_loss = np.convolve(loss, np.ones((period,)) / period, mode='valid')

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Pad with NaNs to match the length of input `close` array
    return np.concatenate([np.full((period,), np.nan), rsi])

# Define the strategy
class MLBasedStrategy(Strategy):
    n1: int = 10
    n2: int = 40
    rsi_period: int = 14
    risk_tolerance: float = 0.02  # Stop-loss tolerance, 2% of the position size

    def init(self):
        close = self.data.close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)
        self.rsi = self.I(RSI, close, self.rsi_period)

        # Prepare data for ML training (SMA1, SMA2, RSI)
        features = pd.DataFrame({
            'SMA1': self.sma1,
            'SMA2': self.sma2,
            'RSI': self.rsi,
            'Close': close
        }).dropna()

        # Define the target: Buy (1), Sell (-1), Hold (0)
        features['Target'] = np.where((features['SMA1'] > features['SMA2']) & (features['RSI'] < 30), 1,
                                      np.where((features['SMA1'] < features['SMA2']) & (features['RSI'] > 70), -1, 0))

        # Split data into training and testing sets
        X = features[['SMA1', 'SMA2', 'RSI']]
        y = features['Target']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

        # Train RandomForest model
        self.model = RandomForestClassifier(n_estimators=100)
        self.model.fit(X_train, y_train)

    def next(self):
        close = self.data.close[-1]
        sma1 = self.sma1[-1]
        sma2 = self.sma2[-1]
        rsi = self.rsi[-1]

        # Machine learning prediction
        prediction = self.model.predict([[sma1, sma2, rsi]])

        # Implement risk management: stop-loss
        if self.position.is_long and close < self.position.entry_price * (1 - self.risk_tolerance):
            self.position.close()
        elif self.position.is_short and close > self.position.entry_price * (1 + self.risk_tolerance):
            self.position.close()

        # Trading decisions based on ML prediction
        if prediction == 1 and not self.position.is_long:
            self.position.close()
            self.buy(size=0.5)
        elif prediction == -1 and not self.position.is_short:
            self.position.close()
            self.sell(size=0.5)

# Initialize the backtester
tester = EmpOrderly(
    cash=1000,
    commission=.0001,
    exclusive_orders=True,
    sdk=sdk,
)

# Load the strategy and data
tester.set_strategy(MLBasedStrategy)
await tester.load_data(
    lookback=12,
    interval=Interval.fifteen_minute,
    asset=PerpetualAssetType.BTC,
)

# Run the backtest
tester.backtest()

# Plot the results
tester.plot(show_price_data=False)
plt.show()


/mnt/Dev Volume/Project/Stackup/empyreal/testing/.venv/lib/python3.11/site-packages/emp_orderly/backtest/backtesting.py:1175: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  self._init_data(self._history)
/mnt/Dev Volume/Project/Stackup/empyreal/testing/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/mnt/Dev Volume/Project/Stackup/empyreal/testing/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/mnt/Dev Volume/Project/Stackup/empyreal/testing/.venv/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassif

[1014/084505.031439:WARNING:chrome_main_linux.cc(80)] Read channel stable from /app/extra/CHROME_VERSION_EXTRA


[1014/084505.298273:WARNING:chrome_main_linux.cc(80)] Read channel stable from /app/extra/CHROME_VERSION_EXTRA
Opening in existing browser session.
